# Modelo de Otorgamiento de Crédito
Este notebook desarrolla un modelo predictivo para estimar la Probabilidad de Incumplimiento (PD) de nuevos solicitantes de crédito. El objetivo es proporcionar una herramienta técnica para la toma de decisiones en el proceso de otorgamiento y la gestión de riesgos.

## 1. Importación de Librerías y Carga de Datos
Cargamos las librerías necesarias para el procesamiento de datos, escalamiento y modelado estadístico. Los datos utilizados provienen de la etapa previa de limpieza.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


apps_model = pd.read_csv('data/clean/apps_model_filtered.csv', parse_dates=['orig_month'], thousands=',')


## 2. Preparación de Datos y División Temporal (Out-of-Time)
Para validar correctamente el modelo, realizamos una división temporal de los datos. 
- Utilizamos el **80% inicial** de los meses para el entrenamiento.
- Reservamos el **20% más reciente** para la prueba (test), simulando cómo se comportaría el modelo con solicitudes futuras.

In [ ]:
apps_sorted = apps_model.sort_values('orig_month')

cutoff = apps_sorted['orig_month'].quantile(0.8)

train = apps_sorted[apps_sorted['orig_month'] <= cutoff]
test  = apps_sorted[apps_sorted['orig_month'] > cutoff]
features = apps_sorted.columns.difference(['customer_id','orig_month','default_12m'])

X_train = train[features]
y_train = train['default_12m']

X_test = test[features]
y_test = test['default_12m']


## 3. Selección de Variables y Escalamiento
Seleccionamos un subconjunto de variables (drivers de riesgo) que tienen una relación teórica y estadística con el incumplimiento. 
Posteriormente, aplicamos un **escalamiento estándar** (StandardScaler) para asegurar que todas las variables tengan la misma escala (media 0 y varianza 1), lo cual es crucial para la interpretación de los coeficientes en una Regresión Logística.

In [ ]:
good_vars = [
    'bureau_score',
    'debt_income',
    'income',
    'infl',
    'prev_delin_24m',
    'rate',
    'utilization'
]
X_train_red = X_train[good_vars]
X_test_red  = X_test[good_vars]
scaler = StandardScaler()

X_train_red_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_red),
    columns=good_vars,
    index=X_train_red.index
)

X_test_red_scaled = pd.DataFrame(
    scaler.transform(X_test_red),
    columns=good_vars,
    index=X_test_red.index
)

## 4. Entrenamiento del Modelo
Ajustamos una **Regresión Logística**. Utilizamos el parámetro `class_weight='balanced'` debido a que los casos de incumplimiento (default) suelen ser una minoría en la población, lo que ayuda al modelo a aprender mejor las características del grupo de riesgo.

In [ ]:
logit_orig = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

logit_orig.fit(X_train_red_scaled, y_train)


## 5. Generación de Probabilidades de Incumplimiento (PD)
Calculamos la probabilidad estimada de default para cada cliente en los conjuntos de entrenamiento y prueba. Esta "score" es la base para la asignación de riesgo.

In [30]:
train['pd_orig'] = logit_orig.predict_proba(X_train_red_scaled)[:, 1]
test['pd_orig'] = logit_orig.predict_proba(X_test_red_scaled)[:, 1]


## 6. Evaluación del Modelo (AUC)
El **AUC (Area Under the Curve)** mide la capacidad del modelo para distinguir entre "buenos" y "malos" pagadores. 
- Un AUC de 0.5 indica un modelo al azar.
- Un AUC > 0.7 se considera generalmente aceptable para modelos de riesgo crediticio.

In [37]:

auc_train = roc_auc_score(y_train, train['pd_orig'])
auc_test = roc_auc_score(y_test, test['pd_orig'])

auc_train, auc_test


(0.6432302729335205, 0.6360678110234061)

## 7. Segmentación por Riesgo (Bucketing)
Dividimos la población de prueba en 10 grupos (deciles) basados en su PD estimada. El grupo 0 contiene a los clientes de menor riesgo y el grupo 9 a los de mayor riesgo.

In [32]:
test['pd_bucket'] = pd.qcut(
    test['pd_orig'],
    10,
    labels=False
)


## 8. Análisis de Métricas y Pérdida Esperada
Calculamos la tasa de default observada frente a la PD promedio en cada decil para verificar la calibración. 
También estimamos la **Pérdida Esperada (Expected Loss)** multiplicando la PD por el ingreso (como proxy de la exposición).

In [33]:
eval_table = (
    test
    .groupby('pd_bucket')
    .agg(
        pd_avg=('pd_orig', 'mean'),
        default_rate=('default_12m', 'mean'),
        total_income=('income', 'sum')
    )
)

eval_table['expected_loss'] = (
        eval_table['pd_avg'] * eval_table['total_income']
)

eval_table


,pd_avg,default_rate,total_income,expected_loss
pd_bucket,,,,
0,0.375650,0.030220,13693634.02,5.144013e+06
1,0.463632,0.035714,13297518.37,6.165155e+06
2,0.504841,0.057851,12384752.14,6.252332e+06
3,0.540174,0.046703,12115963.17,6.544730e+06
4,0.573772,0.054945,11945606.78,6.854056e+06
5,0.603529,0.074380,12236149.46,7.384877e+06
6,0.634051,0.074176,11618439.98,7.366688e+06
7,0.664237,0.090909,11812832.39,7.846520e+06
8,0.704708,0.101648,11331928.77,7.985703e+06


## 9. Interpretación de Coeficientes
Los coeficientes nos indican la dirección y magnitud del impacto de cada variable en el riesgo:
- **Positivo:** A medida que aumenta la variable, aumenta el riesgo de default.
- **Negativo:** A medida que aumenta la variable, disminuye el riesgo (factores protectores).

In [39]:
coef = pd.Series(
    logit_orig.coef_[0],
    index=good_vars
).sort_values()

coef


bureau_score     -0.406951
income           -0.067486
infl              0.060058
utilization       0.114552
debt_income       0.121792
prev_delin_24m    0.126517
rate              0.166644
dtype: float64

## 10. Análisis de Casos Individuales
Visualizamos los clientes con las mayores probabilidades de incumplimiento para entender qué combinación de variables los hace riesgosos.

In [21]:
test[
    ['bureau_score', 'utilization', 'debt_income', 'prev_delin_24m', 'pd_orig']
].sort_values('pd_orig', ascending=False).head(5)


,bureau_score,utilization,debt_income,prev_delin_24m,pd_orig
954,560.8,0.3869,2.5809,0,0.931804
15688,516.9,0.7638,1.9488,1,0.927830
10846,610.4,0.3750,0.6559,4,0.878098
7462,600.0,0.5322,1.6572,0,0.868925
12028,512.6,0.6315,0.4714,0,0.862946


## 11. Análisis de Estrés (Stress Test)
Evaluamos el impacto de un escenario macroeconómico adverso donde la probabilidad de default de toda la cartera se incrementa en un **30%**. Esto ayuda a dimensionar el riesgo en condiciones de crisis.

In [40]:
stress = test.copy()

# Escenario base
stress['pd_base'] = stress['pd_orig']

# Escenario adverso (+30%)
stress['pd_adverse'] = (stress['pd_orig'] * 1.3).clip(0, 1)


## 12. Comparación de Pérdida Esperada bajo Estrés
Calculamos el incremento total en la pérdida esperada al pasar del escenario base al escenario adverso.

In [41]:
stress['el_base'] = stress['pd_base'] * stress['income']
stress['el_adverse'] = stress['pd_adverse'] * stress['income']

stress[['el_base', 'el_adverse']].sum()


el_base       6.988062e+07
el_adverse    9.063346e+07
dtype: float64